<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/notebooks/ann_paper_sim_study_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages for the simulation study

In [ ]:
!pip install annoy

In [ ]:
!pip install pynndescent

In [ ]:
!pip install hnswlib

Load standard modules

In [ ]:
import pandas as pd
import numpy as np
import plotnine as ggplot

Load ann modules

In [ ]:
from sklearn.neighbors import KDTree ## exact search
from annoy import AnnoyIndex